<a href="https://colab.research.google.com/github/muhdlaziem/DR/blob/master/Training_1x2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive


In [0]:
import numpy as np

In [0]:
trainImages = np.load('./diabetic-retinopathy-resized/1x2/trainImages.npy')
testImages = np.load('./diabetic-retinopathy-resized/1x2/testImages.npy')
y_train = np.load('./diabetic-retinopathy-resized/1x2/y_train.npy')
y_test = np.load('./diabetic-retinopathy-resized/1x2/y_test.npy')

In [4]:
print(trainImages.shape)
print(testImages.shape)
print(y_train.shape)
print(y_test.shape)

(7072, 256, 256, 3)
(1769, 256, 256, 3)
(7072, 2)
(1769, 2)


In [5]:
print(trainImages.max())
print(testImages.max())

255
255


In [0]:
# Normalize
trainImages = trainImages/255
testImages = testImages/255

In [7]:
from tensorflow.keras import datasets, layers, models

In [0]:
def convolutional_model():
    
    # create model
    model = models.Sequential()
    model.add(layers.Conv2D(128, (5, 5), activation='relu', input_shape=(256, 256, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (5,5), strides = (1,1), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    # model.add(layers.Dropout(0.5))

    model.add(layers.Conv2D(32, (5,5), strides = (1,1), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    # model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    # model.add(layers.Dropout(0.3))
    model.add(layers.Dense(2, activation='softmax'))

    # model = models.Sequential()
    # model.add(layers.Conv2D(16, (5, 5), activation='relu', input_shape=(256, 256, 3)))
    # model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    # model.add(layers.Conv2D(8, (2, 2), activation='relu'))
    # model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    # model.add(layers.Flatten())
    # model.add(layers.Dense(100, activation='relu'))
    # model.add(layers.Dense(2, activation='softmax'))
    
    model.summary()
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    
    return model

In [9]:
model = convolutional_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 128)     9728      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 64)      204864    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 32)        51232     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________

In [10]:
model.fit(trainImages, y_train, epochs=40, batch_size=100, validation_data=(testImages, y_test))

Train on 7072 samples, validate on 1769 samples
Epoch 1/40
7072/7072 [==============================] - 29s 4ms/sample - loss: 0.5067 - acc: 0.8339 - val_loss: 0.4816 - val_acc: 0.8157
Epoch 2/40
7072/7072 [==============================] - 25s 4ms/sample - loss: 0.4542 - acc: 0.8339 - val_loss: 0.4772 - val_acc: 0.8157
Epoch 3/40
7072/7072 [==============================] - 25s 3ms/sample - loss: 0.4492 - acc: 0.8339 - val_loss: 0.4735 - val_acc: 0.8157
Epoch 4/40
7072/7072 [==============================] - 25s 3ms/sample - loss: 0.4482 - acc: 0.8339 - val_loss: 0.4725 - val_acc: 0.8157
Epoch 5/40
7072/7072 [==============================] - 25s 3ms/sample - loss: 0.4506 - acc: 0.8339 - val_loss: 0.4725 - val_acc: 0.8157
Epoch 6/40
7072/7072 [==============================] - 25s 3ms/sample - loss: 0.4462 - acc: 0.8339 - val_loss: 0.4703 - val_acc: 0.8157
Epoch 7/40
7072/7072 [==============================] - 25s 3ms/sample - loss: 0.4448 - acc: 0.8339 - val_loss: 0.4698 - val_acc: 

In [11]:
scores = model.evaluate(testImages, y_test, verbose=0)
print('Accuracy: {}% \n Error: {}'.format(scores[1], 1 - scores[1])) 

Accuracy: 0.8004522323608398% 
 Error: 0.19954776763916016


In [12]:
from sklearn import metrics
y_pred = model.predict(testImages)
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
matrix

array([[1372,   71],
       [ 282,   44]])

In [13]:
model.save("training_1x2_1.h5")
print("Saved model to disk")

Saved model to disk
